# Setup

In [28]:
import paho.mqtt.client as mqtt
import ssl
from pathlib import Path  
from dotenv import load_dotenv
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#SECRET KEYS
env_path = Path('.') / 'config.env'
load_dotenv(dotenv_path=env_path)

PASSWORD = os.getenv("API_KEY")
TOPIC = os.getenv("TOPIC")
USERNAME = os.getenv("POZYX_USERNAME")

# Define Connection

In [25]:
host = "mqtt.cloud.pozyxlabs.com"
port = 443
topic = TOPIC
username = USERNAME  
password = PASSWORD  

# Start Client

In [26]:
client = mqtt.Client(transport="websockets")
client.username_pw_set(username, password=password)

# sets the secure context, enabling the WSS protocol
client.tls_set_context(context=ssl.create_default_context())


payloads = []
def on_connect(client, userdata, flags, rc):
    print(mqtt.connack_string(rc))

# Callback triggered by a new Pozyx data packet
def on_message(client, userdata, msg):
    #print("Positioning update:", msg.payload.decode())
    payloads.append(msg.payload.decode())
    
def on_subscribe(client, userdata, mid, granted_qos):
    print("Subscribed to topic!")


# set callbacks
client.on_connect = on_connect
client.on_message = on_message
client.on_subscribe = on_subscribe
client.connect(host, port=port)
client.subscribe(topic)

# works blocking, other, non-blocking, clients are available too.
client.loop_forever()

Connection Accepted.
Subscribed to topic!


KeyboardInterrupt: 

# Data Wrangling

In [30]:
json_list = []
for i in payloads:
    json_list.append(json.loads(i))

In [34]:
df = pd.DataFrame(json_list)

In [48]:
coords = []

for two_tags in json_list:
    tag_1_coords = two_tags[0]['data']['coordinates']
    coords.append(tag_1_coords)

In [50]:
c_df = pd.DataFrame(coords)

In [51]:
c_df

,x,y,z
0,1781,850,685
1,1780,852,687
2,1779,853,688
3,1781,856,690
4,1782,858,690
...,...,...,...
88,1980,2774,1151
89,1898,2746,1157
90,1735,2629,1154
91,1660,2539,1143


In [27]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(c_df['x'], c_df['y'], c_df['z'])

NameError: name 'plt' is not defined